In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# Robust Aristocrat (monoalphabetic substitution) solver — no manual seeds.
# - Prompts you to paste the ciphertext (multi-line); press Enter on an empty line to finish
# - Auto-seeds common patterns (THE, THAT)
# - Simulated annealing + restarts + steepest-ascent polish
# - Fitness: smoothed tri+bigram log-likelihood + small word/punct bonuses

import math, random, re
from collections import Counter
from string import ascii_uppercase as AZ

# ---------- English n-grams (compact, smoothed) ----------
_BIGRAM_COUNTS = {
    "TH":1800000,"HE":1300000,"IN":1000000,"ER":950000,"AN":920000,"RE":900000,"ND":880000,
    "ON":860000,"EN":850000,"AT":840000,"OU":820000,"ED":800000,"HA":760000,"TO":750000,
    "OR":740000,"IT":730000,"IS":720000,"HI":700000,"ES":690000,"NG":680000,"ST":670000,
    "AR":660000,"TE":650000,"SE":640000,"ME":620000,"AL":610000,"LE":600000,"OF":590000,
    "DE":580000,"SA":570000,"SI":560000,"VE":550000,"RA":540000,"NO":530000,"EA":520000,
    "TI":510000,"RO":500000,"IC":490000,"NE":480000,"LI":470000,"LA":460000,"EL":450000,
    "TA":440000,"IO":430000,"RI":420000,"CO":410000,"BE":400000,"AS":390000,"CE":380000,
    "UR":370000,"US":360000,"PR":350000,"MA":340000,"CH":330000,"TR":320000,"DI":310000,
    "UN":300000,"LO":290000,"WH":220000,"SH":260000,"EE":250000,"OO":180000,"LL":210000,
    "SS":205000,"TT":195000,"NN":190000,"RR":200000,"PP":90000,"GH":110000,"QU":100000,
    "AA":5000,
}
_TRIGRAM_COUNTS = {
    "THE":180000,"AND":120000,"ING":110000,"HER":80000,"HAT":70000,"ION":65000,"ERE":60000,
    "ENT":58000,"THA":57000,"NTH":56000,"WAS":55000,"ETH":54000,"FOR":53000,"DTH":50000,
    "HES":49000,"EST":48000,"HIS":47000,"ERS":46000,"VER":45000,"ATI":44000,"ALL":43000,
    "TER":42000,"TIO":41000,"WIT":40000,"NOT":39000,
}
_BG_FLOOR, _TG_FLOOR = 5000, 200
_BG_TOTAL = sum(_BIGRAM_COUNTS.values()) + _BG_FLOOR * (26*26 - len(_BIGRAM_COUNTS))
_TG_TOTAL = sum(_TRIGRAM_COUNTS.values()) + _TG_FLOOR * (26*26*26 - len(_TRIGRAM_COUNTS))
BIGRAM_LP  = {a+b  : math.log(_BIGRAM_COUNTS.get(a+b, _BG_FLOOR) / _BG_TOTAL) for a in AZ for b in AZ}
TRIGRAM_LP = {a+b+c: math.log(_TRIGRAM_COUNTS.get(a+b+c, _TG_FLOOR) / _TG_TOTAL) for a in AZ for b in AZ for c in AZ}

_COMMON_WORDS = {
    "THE","AND","TO","OF","IN","THAT","IT","IS","FOR","ON","WITH","AS","YOU","BE",
    "AT","BY","ARE","THIS","NOT","BUT","HAVE","FROM","OR","ONE","ALL","WE","CAN",
    "THERE","THEY","AN","WHICH","IF","WHEN","WHO","WILL","ABOUT","THEIR","THEN",
    "SO","WAS","HE","SHE","I"
}

# ---------- Helpers ----------
def normalize(s: str) -> str:
    return re.sub(r"[^A-Z ,.'?!:-]", "", s.upper())

def letters_only(s: str) -> str:
    return re.sub(r"[^A-Z]", "", s.upper())

def apply_key(text: str, keymap: dict) -> str:
    return ''.join(keymap.get(ch, '?') if 'A' <= ch <= 'Z' else ch for ch in text)

def keymap_from_perm(perm: str) -> dict:
    return {c: p for c, p in zip(AZ, perm)}  # cipher A..Z -> plain

def invert_map(d: dict) -> dict:
    return {v:k for k, v in d.items()}

def swap(perm: str, i: int, j: int) -> str:
    if i == j: return perm
    p = list(perm); p[i], p[j] = p[j], p[i]
    return ''.join(p)

def cycle3(perm: str, i: int, j: int, k: int) -> str:
    if i==j or j==k or i==k: return perm
    p = list(perm); p[i], p[j], p[k] = p[k], p[i], p[j]
    return ''.join(p)

def word_pattern(word: str) -> str:
    m, out, n = {}, [], 0
    for ch in word:
        if ch not in m:
            m[ch] = chr(ord('A') + n); n += 1
        out.append(m[ch])
    return ''.join(out)

# ---------- Seeding (auto only) ----------
def freq_seed(ciphertext: str) -> str:
    letters = [c for c in ciphertext if 'A' <= c <= 'Z']
    freq = Counter(letters)
    english = list("ETAOINSHRDLCUMWFGYPBVKJXQZ")
    sorted_cipher = [x for x,_ in freq.most_common()]
    mapping = {}
    for i, c in enumerate(sorted_cipher):
        if i < len(english): mapping[c] = english[i]
    used = set(mapping.values())
    remaining = [ch for ch in english if ch not in used] + [ch for ch in AZ if ch not in used and ch not in english]
    for c in AZ:
        if c not in mapping: mapping[c] = remaining.pop(0)
    return ''.join(mapping[c] for c in AZ)

def force_word_map(perm: str, cipher_word: str, plain_word: str) -> str:
    """Force cipher_word -> plain_word by swapping outputs in perm (keeps bijection)."""
    assert len(cipher_word) == len(plain_word)
    km = keymap_from_perm(perm); inv = invert_map(km)
    for c, p in zip(cipher_word, plain_word.upper()):
        if km[c] == p: continue
        i = AZ.index(c); j = AZ.index(inv[p])
        perm = swap(perm, i, j)
        km = keymap_from_perm(perm); inv = invert_map(km)
    return perm

def auto_seed(ciphertext: str, perm: str) -> str:
    words = re.findall(r"[A-Z]+", ciphertext)
    # THE: most frequent 3-letter with all distinct letters
    c3 = Counter(w for w in words if len(w)==3 and len(set(w))==3).most_common(1)
    if c3: perm = force_word_map(perm, c3[0][0], "THE")
    # THAT: frequent ABAC 4-letter word
    c4 = Counter(w for w in words if len(w)==4 and word_pattern(w)=="ABAC")
    if c4:
        w4,_ = c4.most_common(1)[0]
        perm = force_word_map(perm, w4, "THAT")
    return perm

# ---------- Fitness ----------
def score_plain(plain: str) -> float:
    txt = letters_only(plain)
    if len(txt) < 2: return -1e9
    s_bg = sum(BIGRAM_LP [txt[i:i+2]] for i in range(len(txt)-1))
    s_tg = sum(TRIGRAM_LP[txt[i:i+3]] for i in range(len(txt)-2))
    score = 0.55 * s_tg + 0.45 * s_bg
    # gentle bonuses & penalties
    words = re.findall(r"[A-Z]+", plain.upper())
    score += 1.2 * sum(1 for w in words if w in _COMMON_WORDS)
    score += 0.15 * plain.count(", ") + 0.25 * plain.count(". ")
    score -= 1.0  * plain.count("Q ")        # Q rarely ends a word (no following U)
    return score

# ---------- Solver ----------
def solve(ciphertext: str, restarts=64, steps=30000, t0=3.2, t1=0.04, patience=3500, rng_seed=42):
    if rng_seed is not None:
        random.seed(rng_seed)
    if not any('A' <= c <= 'Z' for c in ciphertext):
        ident = {c:c for c in AZ}
        return ciphertext, ident, 0.0

    best_score, best_perm = -1e99, None

    for _ in range(restarts):
        perm = auto_seed(ciphertext, freq_seed(ciphertext))

        cur_perm = perm
        cur_plain = apply_key(ciphertext, keymap_from_perm(cur_perm))
        cur_score = score_plain(cur_plain)
        stalled = 0

        for s in range(steps):
            T = t0 + (t1 - t0) * (s / max(1, steps-1))
            # mixed proposals
            if random.random() < 0.8:
                i, j = random.randrange(26), random.randrange(26)
                trial = swap(cur_perm, i, j)
            else:
                i, j, k = random.randrange(26), random.randrange(26), random.randrange(26)
                trial = cycle3(cur_perm, i, j, k)

            trial_plain = apply_key(ciphertext, keymap_from_perm(trial))
            sc = score_plain(trial_plain)
            delta = sc - cur_score
            if delta >= 0 or math.exp(delta / max(T, 1e-9)) > random.random():
                cur_perm, cur_score, cur_plain = trial, sc, trial_plain
                stalled = 0
            else:
                stalled += 1

            if stalled > patience:
                cur_perm = auto_seed(ciphertext, freq_seed(ciphertext))
                cur_plain = apply_key(ciphertext, keymap_from_perm(cur_perm))
                cur_score = score_plain(cur_plain)
                stalled = 0

        # polish: steepest-ascent over all pair swaps
        improved = True
        while improved:
            improved = False
            base_perm, base_score = cur_perm, cur_score
            best_local_perm, best_local_score = base_perm, base_score
            for i in range(26):
                for j in range(i+1, 26):
                    trial = swap(base_perm, i, j)
                    sc = score_plain(apply_key(ciphertext, keymap_from_perm(trial)))
                    if sc > best_local_score:
                        best_local_score, best_local_perm = sc, trial
                        improved = True
            if improved:
                cur_perm, cur_score = best_local_perm, best_local_score

        if cur_score > best_score:
            best_score, best_perm = cur_score, cur_perm

    return apply_key(ciphertext, keymap_from_perm(best_perm)), keymap_from_perm(best_perm), best_score

# ---------- I/O ----------
def read_paste() -> str:
    print("Paste your ciphertext (multi-line OK). Press Enter on an empty line to solve.\n")
    lines = []
    while True:
        try:
            line = input()
        except EOFError:
            break
        if line.strip() == "":
            break
        lines.append(line)
    return "\n".join(lines)

if __name__ == "__main__":
    raw = read_paste()
    if not raw.strip():
        print("No input received. Exiting.")
        raise SystemExit(0)

    C = normalize(raw)
    plaintext, keymap, score = solve(C, restarts=64, steps=30000, rng_seed=42)

    print("\n=== DECODED TEXT ===\n")
    print(plaintext)
    print("\n=== KEY (cipher -> plain) ===")
    print("CIPHER:", " ".join(AZ))
    print("PLAIN :", " ".join(keymap[c] for c in AZ))
    print(f"\nScore: {score:.2f}")
